# Quick Example for AutoDiff w/ Mass

Just computes generalized forces for a simple body.

We create a simple `MultibodyPlant` (`MultibodyPlant_[float]`). This means you *could*
parse it from an URDF / SDFormat file.
We then convert the system to `AutoDiffXd`, and set the parameters we desire. In this case,
we want $\frac{\partial{f}}{\partial{m}}$, where $f$ is just some arbitrary expression.

In our case, we choose $f$ to be generalized forces at the default / home configuration.

In [1]:
from pydrake.all import (
    MultibodyPlant_,
    MultibodyPlant,
    AutoDiffXd,
    SpatialInertia,
    UnitInertia,
)

In [2]:
plant = MultibodyPlant(time_step=0.0)
body = plant.AddRigidBody(
    "body",
    M_BBo_B=SpatialInertia(
        mass=2.0,
        p_PScm_E=[0, 0, 0],
        G_SP_E=UnitInertia(0.1, 0.1, 0.1),
    ),
)
plant.Finalize()

plant_ad = plant.ToScalarType[AutoDiffXd]()
body_ad = plant_ad.get_body(body.index())
context_ad = plant_ad.CreateDefaultContext()

In [3]:
m = AutoDiffXd(2.0, [1.0])
body_ad.SetMass(context_ad, m)

The value should just be $m \cdot g$, and the derivative should then be $g$.

In [4]:
qwxyz_xyz = plant_ad.CalcGravityGeneralizedForces(context_ad)
xyz = qwxyz_xyz[-3:]
z = xyz[2]
print(z.value())
print(z.derivatives())

-19.62
[-9.81]
